In [1]:
import sys
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols

default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output


In [4]:
read_cc = True #and False
read_tb = True #and False

In [5]:
cc = CasosConfirmados()
if read_cc:
    cc.read_excel()
    cc.save()
else:
    cc.load()

data: /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados.pkl
arquivo: /home/vinnylg/projects/covid19datascience/input/Casos confirmados PR.xlsx
reading from Casos confirmados PR.xlsx time elepsed:00:03:41
saving Casos Confirmados to pkl time elepsed:00:00:05


In [6]:
casosc = cc.get_casos()
casosc.loc[casosc['obito']!='SIM','obito'] = 'NAO'
print(f"casosc len: {len(casosc)}\n")
casosc.groupby('obito')[['sexo']].count()

casosc len: 1046917

['ordem' 'identificacao' 'ibge7' 'rs_res_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'rs' 'laboratorio' 'dt_diag' 'comunicacao' 'is'
 'obito' 'data_obito' 'excluir' 'hash' 'hash_less' 'hash_more'
 'hash_atend' 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


,sexo
obito,
NAO,1021714
SIM,25203


In [7]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
casost.groupby('obito')[['sexo']].count()

Reading TbPacientes time elepsed:00:04:33
casost len: 1046917

['identificacao' 'ibge_res_pr' 'ibge_atend_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'laboratorio' 'dt_diag' 'dt_notificacao'
 'dt_inicio_sintomas' '_dia_isolamento' 'dt_viagem_retorno' 'local_viagem'
 'exposicao' 'data_de_internamento' 'local' 'clinico_uti' 'dt_alta'
 'obito' 'dt_obito' 'muni_res_pr' 'hist_viagem' 'casos_novos' 'telefone'
 'dt_atualizacao' 'fonte' 'status' 'eas' 'privado_sus' 'uti'
 'dt_internamento' 'evolucao' 'dt_evolucao' 'temp_permanencia'
 'cod_laboratorio' 'idade_original' 'dt_com_obito' 'dt_com_recuperado'
 'idade_mais' 'idade_menos' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


,sexo
obito,
NAO,1021714
SIM,25203


In [9]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [10]:
del casosc['identificacao']

In [11]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

25200
1021717


In [12]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1045643
1892


In [13]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1047537
0


In [14]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1047537
0


In [15]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

1047537
0


,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [16]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [17]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag'].values==ordem_duplicado['hash_diag'].values)

745 745


True

In [18]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')
id_duplicado.loc[~id_duplicado['hash_diag'].isin(ordem_duplicado['hash_diag'])]

,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [19]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

1047537
casos: 1046917
obitos: 25203


In [85]:
try:
    casosct = casosct_bkp.copy()
except:
    casosct_bkp = casosct.copy()
    
# casosct = casosct.sort_values(['comunicacao','nome','idade']).copy()
casosct = casosct.sort_values(['ordem']).copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

# municipios = Municipios()
# casosct = pd.merge(casosct,municipios[['ibge7','regional','macro','uf','municipio']].rename(columns={'municipio':'municipio_pr'}),how='left',on='ibge7')

# casosct.loc[casosct['rs']==99, 'macro'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'regional'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'uf'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'municipio_pr'] = 'Fora'

# macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
# macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
# casosct['macro'] = casosct['macro'].apply(lambda x: macros_labels_idx[x])

casosct.columns

Index(['ordem', 'ibge7', 'rs_res_pr', 'nome', 'sexo', 'idade', 'mun_resid',
       'mun_atend', 'rs', 'laboratorio', 'dt_diag', 'comunicacao', 'is',
       'obito', 'data_obito', 'excluir', 'hash', 'hash_less', 'hash_more',
       'hash_atend', 'hash_less_atend_x', 'hash_more_atend_x', 'hash_diag',
       'hash_obito', 'identificacao', 'ibge_atend_pr', 'dt_com_obito',
       'status', 'dt_com_recuperado', 'hash_less_atend_y', 'hash_more_atend_y',
       'evolucao', 'data_evolucao', 'data_com_evolucao'],
      dtype='object')

In [86]:
casosc = cc.get_casos()
casosct.loc[~casosct['ordem'].isin(casosc['ordem'])]

,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y,evolucao,data_evolucao,data_com_evolucao


In [87]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios


casosct['uf'] = ''
casosct = casosct[['ordem','identificacao','uf','ibge7','ibge_atend_pr','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','excluir']]

In [88]:
casos_fora = casosct.loc[casosct['ibge7']==9999999].copy()
casos_fora['uf'] = casos_fora['mun_resid'].apply(lambda x: x.split('/')[-1])
casos_fora['mun_resid'] = casos_fora['mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casos_fora.drop(columns=['ibge7']),municipios[['ibge7','mun_resid','uf']], on=['uf','mun_resid'], how='left')
casos_fora.loc[casos_fora['ibge7'].isna(),'excluir'] = 'SIM'
casos_fora.loc[casos_fora['ibge7'].isna(),'ibge7'] = 9999999
casos_fora

,ordem,identificacao,uf,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,excluir,ibge7
0,9,13673,SP,4106902,ITALO FERNANDO TROMBINI FILHO,M,28,SAO PAULO,CURITIBA,99,GENOPRIMER,2020-03-16,2020-03-16,NaT,1,2020-08-05,2020-08-05,NaN,3550308.0
1,65,19531,DF,4106902,LUCIANO PUCHALSKI,M,59,BRASILIA,CURITIBA,99,LACEN,2020-03-23,2020-03-24,2020-03-13,3,NaT,NaT,NaN,5300108.0
2,79,15204,SP,4103701,JOAO HENRIQUE FONZAR BEGNINI,M,37,SAO PAULO,CAMBE,99,LACEN,2020-03-25,2020-03-25,2020-03-17,3,NaT,NaT,NaN,3550308.0
3,177,19629,RJ,4114351,LUCINEIA ALVES DE ANHAIA,F,38,NITEROI,MANFRINOPOLIS,99,LACEN,2020-03-31,2020-03-31,2020-03-22,3,NaT,NaT,NaN,3303302.0
4,220,25622,SP,4106902,PAULO FRANCISCO NETO,M,25,SAO PAULO,CURITIBA,99,LACEN,2020-03-31,2020-04-01,2020-03-12,1,2020-08-05,2020-08-05,NaN,3550308.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5745,982844,1067868,RS,4119905,ARLI DE SOUZA SEVERGNINI,M,60,BARRA DO RIBEIRO,PONTA GROSSA,99,Teste Rápido IgG (Imunocromatografia),2021-03-27,2021-05-09,2021-03-25,3,NaT,NaT,NaN,4301909.0
5746,982919,1067943,MT,4113700,CARLOS EDUARDO DE SOUZA,M,51,CONFRESA,LONDRINA,99,Teste Rápido Antígeno (Imunocromatografia),2021-05-08,2021-05-09,2021-05-05,3,NaT,NaT,NaN,5103353.0
5747,984355,1069379,SC,4109401,ROMERO RODRIGUES DE OLIVEIRA,M,37,ITAJAI,GUARAPUAVA,99,Teste Rápido Antígeno (Imunocromatografia),2021-05-08,2021-05-09,2021-05-05,3,NaT,NaT,NaN,4208203.0
5748,984556,1069580,SC,4118501,TEREZA TAVARES DA ROCHA,F,77,SAO LOURENCO DO OESTE,PATO BRANCO,99,Teste Rápido Antígeno (Imunocromatografia),2021-04-09,2021-05-09,2021-04-06,2,2021-04-11,2021-05-09,NaN,4216909.0


In [89]:
casosct = casosct.drop(index=casos_fora.index)
casosct['uf'] = "PR"
casosct = pd.concat([casosct,casos_fora])
casosct['ibge7'] = casosct['ibge7'].apply(int)
casosct= casosct.sort_values('ordem')
casosct

,ordem,identificacao,uf,ibge7,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,excluir
25200,1,8838,PR,4105508,4105508,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,13,LACEN,2020-03-12,2020-03-12,NaT,1,NaT,NaT,NaN
25201,2,10932,PR,4106902,4106902,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,NaN
25202,3,10933,PR,4106902,4106902,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,NaN
25203,4,19775,PR,4106902,4106902,LUIZ ANTONIO CAXAMBU FREITAS,M,54,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-11,2020-03-12,2020-03-09,1,NaT,NaT,NaN
25204,5,22343,PR,4106902,4106902,MARIANA CAXAMBU FREITAS,F,25,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-11,2020-03-12,2020-03-01,1,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045638,1047087,1132112,PR,4125506,4125506,ZORAIA REGINA DE ALMEIDA TELLES BAIFFUS,F,57,SAO JOSE DOS PINHAIS,SAO JOSE DOS PINHAIS,2,"COVID-19, Biologia Molecular",2021-05-18,2021-05-20,2021-05-13,3,NaT,NaT,NaN
1045639,1047088,1132113,PR,4104907,4104907,ZORAIDE CORREA DA SILVA,F,60,CASTRO,CASTRO,3,"COVID-19, Biologia Molecular",2021-05-19,2021-05-20,2021-05-16,3,NaT,NaT,NaN
1045640,1047089,1132114,PR,4119905,4119905,ZORANDI RODRIGUES BURAK,F,54,PONTA GROSSA,PONTA GROSSA,3,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-05-19,2021-05-20,NaT,3,NaT,NaT,NaN
1045641,1047090,1132115,PR,4122701,4106902,ZULEIDE DE LIRA CECILIA,F,63,SABAUDIA,CURITIBA,16,Teste Rápido Antígeno (Imunocromatografia),2021-05-19,2021-05-20,2021-05-17,3,NaT,NaT,NaN


In [90]:
#####
## novos casos confirmados
#

casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge7'].apply(lambda x: str(x)[:6])
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(lambda x: str(x)[:6])

casosct['hash'] = (casosct['nome'].apply(normalize_hash) +
                  casosct['idade'].astype(str) +
                  casosct['ibge_resid'].astype(str))

casosct['hash_less'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x - 1)) +
                        casosct['ibge_resid'].astype(str))

casosct['hash_more'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x + 1)) +
                        casosct['ibge_resid'].astype(str))

casosct['hash_atend'] = (casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].astype(str) +
                        casosct['ibge_atend'].astype(str))

casosct['hash_less_atend'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x - 1)) +
                        casosct['ibge_atend'].astype(str))

casosct['hash_more_atend'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x + 1)) +
                        casosct['ibge_atend'].astype(str))


casosct['hash_diag'] = (casosct['nome'].apply(normalize_hash) + casosct['dt_diag'].apply(date_hash))

casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna(), 'hash_obito'] = casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna()].apply(
        lambda row: normalize_hash(row['nome']) + date_hash(row['data_com_evolucao']), axis=1
)

casosct = casosct.rename(columns={'uf':'uf_resid'})

casosct['ibge_resid'] = casosct['ibge_resid'].apply(int)
casosct['ibge_atend'] = casosct['ibge_resid'].apply(int)

casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag','hash_obito']]
casosct

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito
25200,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,NaT,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
25201,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020,NaN
25202,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
25203,19775,-1,PR,410690,410690,LUIZ ANTONIO CAXAMBU FREITAS,M,54,GENOPRIMER,2020-03-11,2020-03-12,2020-03-09,1,NaT,NaT,LUIZANTONIOCAXAMBUFREITAS54410690,LUIZANTONIOCAXAMBUFREITAS53410690,LUIZANTONIOCAXAMBUFREITAS55410690,LUIZANTONIOCAXAMBUFREITAS54410690,LUIZANTONIOCAXAMBUFREITAS53410690,LUIZANTONIOCAXAMBUFREITAS55410690,LUIZANTONIOCAXAMBUFREITAS11032020,NaN
25204,22343,-1,PR,410690,410690,MARIANA CAXAMBU FREITAS,F,25,GENOPRIMER,2020-03-11,2020-03-12,2020-03-01,1,NaT,NaT,MARIANACAXAMBUFREITAS25410690,MARIANACAXAMBUFREITAS24410690,MARIANACAXAMBUFREITAS26410690,MARIANACAXAMBUFREITAS25410690,MARIANACAXAMBUFREITAS24410690,MARIANACAXAMBUFREITAS26410690,MARIANACAXAMBUFREITAS11032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045638,1132112,-1,PR,412550,412550,ZORAIA REGINA DE ALMEIDA TELLES BAIFFUS,F,57,"COVID-19, Biologia Molecular",2021-05-18,2021-05-20,2021-05-13,3,NaT,NaT,ZORAIAREGINADEALMEIDATELLESBAIFFUS57412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS56412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS58412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS57412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS56412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS58412550,ZORAIAREGINADEALMEIDATELLESBAIFFUS18052021,NaN
1045639,1132113,-1,PR,410490,410490,ZORAIDE CORREA DA SILVA,F,60,"COVID-19, Biologia Molecular",2021-05-19,2021-05-20,2021-05-16,3,NaT,NaT,ZORAIDECORREADASILVA60410490,ZORAIDECORREADASILVA59410490,ZORAIDECORREADASILVA61410490,ZORAIDECORREADASILVA60410490,ZORAIDECORREADASILVA59410490,ZORAIDECORREADASILVA61410490,ZORAIDECORREADASILVA19052021,NaN
1045640,1132114,-1,PR,411990,411990,ZORANDI RODRIGUES BURAK,F,54,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-05-19,2021-05-20,NaT,3,NaT,NaT,ZORANDIRODRIGUESBURAK54411990,ZORANDIRODRIGUESBURAK53411990,ZORANDIRODRIGUESBURAK55411990,ZORANDIRODRIGUESBURAK54411990,ZORANDIRODRIGUESBURAK53411990,ZORANDIRODRIGUESBURAK55411990,ZORANDIRODRIGUESBURAK19052021,NaN
1045641,1132115,-1,PR,412270,412270,ZULEIDE DE LIRA CECILIA,F,63,Teste Rápido Antígeno (Imunocromatografia),2021-05-19,2021-05-20,2021-05-17,3,NaT,NaT,ZULEIDEDELIRACECILIA63412270,ZULEIDEDELIRACECILIA62412270,ZULEIDEDELIRACECILIA64412270,ZULEIDEDELIRACECILIA63410690,ZULEIDEDELIRACECILIA62410690,ZULEIDEDELIRACECILIA64410690,ZULEIDEDELIRACECILIA19052021,NaN


In [93]:
from bulletin.utils.timer import Timer
with Timer('reading from Casos confirmados PR.xlsx'):
    casosct.to_excel('ultimo_casos_confirmados_inteiro.xlsx',index=False)

reading from Casos confirmados PR.xlsx time elepsed:00:05:56
